In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [2]:
import pandas as pd
from pathlib import Path
import numpyro
import jax
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from numpyro.contrib.funsor import config_enumerate, infer_discrete
from src import preprocessing
from scipy.stats import gaussian_kde

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [3]:
raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")
data = preprocessing.preprocess(raw_data)
elections_dict = preprocessing.get_elections_dict(data)

/tmp/ipykernel_1009642/3180180344.py:1: DtypeWarning: Columns (2,3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "Local_2018.csv")


In [11]:
county_ids = list(elections_dict.keys())
example_county_id = county_ids[400]
example_county_elections = elections_dict[example_county_id]
top_two_elections = {}
for election_id, election in example_county_elections.items():
    top_two_elections[election_id] = election.sort_values(by = "votes", ascending=False).iloc[:2]

In [27]:
def model_basic(num_voters=None, votes=None):
    p = numpyro.sample("p", dist.Beta(4, 4))
    numpyro.sample(
        "observed_votes", dist.Binomial(total_count=num_voters, probs=p), obs=votes
    )

In [28]:
ex_election = top_two_elections["CITY COUNCIL - COLUMBIA"]
num_voters = ex_election["votes"].sum()
votes_0 = ex_election.iloc[0]["votes"]

In [29]:
# Start from this source of randomness. We will split keys for subsequent operations.
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

# Run NUTS.
kernel = NUTS(model_basic)
num_samples = 2_000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_key_,
    num_voters = num_voters,
    votes = votes_0
)
mcmc.print_summary()
samples_basic = mcmc.get_samples()

sample: 100%|██████████| 3000/3000 [00:04<00:00, 656.16it/s, 1 steps of size 1.06e+00. acc. prob=0.93]


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         p      0.50      0.01      0.50      0.48      0.52    581.12      1.00

Number of divergences: 0


In [49]:
def model_basic_batched(num_voters_array=None, votes_array=None):
    # with numpyro.plate("election", len(num_voters_array)) as ind:
    p = numpyro.sample("p", dist.Beta(jnp.full(len(num_voters_array), 4), jnp.full(len(num_voters_array), 4)))
    numpyro.sample(
        "observed_votes",
        dist.Binomial(total_count=num_voters, probs=p),
        obs=votes_array,
    )

In [50]:
num_voters_list = []
votes_list = []
for key, election in top_two_elections.items():
    num_voters_list.append(election["votes"].sum())
    votes_list.append(election.iloc[0]["votes"])

In [51]:
# Run NUTS.
kernel = NUTS(model_basic_batched)
num_samples = 2_000
mcmc = MCMC(kernel, num_warmup=1000, num_samples=num_samples)
mcmc.run(
    rng_key_,
    jnp.array(num_voters_list),
    jnp.array(votes_list)
)
mcmc.print_summary()
samples_basic = mcmc.get_samples()

/tmp/ipykernel_1009642/619616848.py:5: UserWarning: Site observed_votes: Out-of-support values provided to log prob method. The value argument should be within the support.
  mcmc.run(


RuntimeError: Cannot find valid initial parameters. Please check your model again.

In [37]:
gaussian_kde(samples_basic["p"])(0.5) / (num_voters + 1)

array([0.02034278])